## LLM

In [18]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
azure_endpoint = os.environ.get('AZURE_ENDPOINT')

llm_config_gemini15 = [
    {
        "model": "gemini-1.5-flash", 
        "api_key": GOOGLE_API_KEY, 
        "cache_seed": 41, 
        "temperature": 0,
        "api_type": "google",
    }
]

llm_config_gemini15Pro = [
    {
        "model": "gemini-1.5-pro", 
        "api_key": GOOGLE_API_KEY, 
        "cache_seed": 41, 
        "temperature": 0,
        "api_type": "google",
    }
]

openai_config_list = [
  {
    "model": "gpt-4.1",
    "api_key": OPENAI_API_KEY,
    "base_url": azure_endpoint,

  },
  {
    "model": "gpt-4-turbo",
    "api_key": OPENAI_API_KEY,
    "base_url": azure_endpoint,
  }
]

In [19]:
llm_config_gemini = {
        "config_list": llm_config_gemini15, #gemini-1.5-flash
    }

llm_config_geminiPro = {
        "config_list": llm_config_gemini15Pro,
    }

llm_config_openai = {
        "config_list": openai_config_list,
    }

llm_config = llm_config_openai

In [20]:
with open("optics_help.txt", "r") as f:
        optics_help = f.read()

with open("optics_best.txt", "r") as f:
        optics_best = f.read()


In [ ]:
from autogen import ConversableAgent
from autogen import GroupChat
from autogen import GroupChatManager
from autogen import UserProxyAgent
from autogen import AssistantAgent
from autogen.coding import LocalCommandLineCodeExecutor
from autogen import register_function

Coder = ConversableAgent(
    name="Coder",
    system_message="""You are a Python developer. You write Python scripts to collect data from networking devices. 
    Always provide the complete code to your team. Do not generate placeholder code. 
     """,
    llm_config=llm_config,
)

Netengineer = ConversableAgent(
    name="Netengineer",
    system_message="""You are a network engineer. You outline the plan for the team. You review and enhance Python scripts. You execute and fix script errors.
    You parse the command instructions to extract the correct commands to run on a device, and update the script accordingly.
    You run scripts to collect network device data and provide instructions to data analysts.
     """,
    llm_config=llm_config,
)

Analyst = AssistantAgent(
    name="Analyst",
    system_message="""You are a data analyst. You take the network outputs and apply best practices to generate summaries and recommendations.
    """,
    llm_config=llm_config # llm_config_claude,
)

Reporter = AssistantAgent(
    name="Reporter",
    system_message="""You are a networking reporter. You write network report based on recommendations.
    """,
    llm_config=llm_config,
)

user_proxy = UserProxyAgent(
    name = "user_proxy",
    human_input_mode = "ALWAYS",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "") and x.get(
        "content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        # the executor to run the generated code
        "executor": LocalCommandLineCodeExecutor(work_dir="tmp_dir"),
    },
    
)



group_chat = GroupChat(
    agents = [user_proxy, Netengineer, Coder, Analyst, Reporter],
    messages = [],
    max_round = 6,
    send_introductions= True,
    speaker_selection_method = "auto",
)

group_chat_manager = GroupChatManager(
    groupchat = group_chat,
    llm_config = llm_config,
)

chat_result = user_proxy.initiate_chat(
    group_chat_manager,
    message=f"""You are a team to collaborate on a networking task. 
    
    Break the task into the following 4 key steps: 

    Step 1: collect optics data and save the data to a file, 
    Devices are Cisco IOS-XR routers (provided by routers.csv file; 
    which is a device file, and the order of data is described by the first row of the file,
    (hostname,ip_address,username,password,device_type). You connect to each device by its IP address using the device type provided.
    
    Follow these 2 steps to collect the optics data on each device:
    Step 1.1: Collect all instances of optics IDs by running this command as they vary from device to device: 
    show running | include controller Optics
    As an example, the following output shows 3 instances of optics in a device, Optics ID follows Optics, an example ID is 0/0/0/2:
    controller Optics0/0/0/2
    controller Optics0/0/0/4
    controller Optics0/0/0/10

    Step 1.2: use the following two commands for each instance of optics IDs on each device, and collect the command output for each:
    show controller optics <ID>
    show controller CoherentDSP <ID>

    Step 2: analyze the optics data, apply optics best practices, generate recommendations for each optics in each device:
    Parse the command output to extract information for each optics ID and device using the following help. 
    {optics_help}. 
    Capture controller states, alarms, Q margin for each optics per device.
    Use the best practice rules {optics_best} to analyze the data and generate recommendations.
    Save the recommendations to a file.
    
    Step 3: Combine all scripts into a single script, save it to a script file, and execute the script using the device file. 
    Task is completed when the entire script is executed successfully and a final report is generated. 

    Step 4: generate a final report in markdown format and save it to a file. 
    Write a final report to summarize the recommendations for each Optics ID per device in a tabular format.
    The tabular report starts with a device name (using the real hostname and IP address but anonymized to hide confidential information)
    and optics ID, for each show the collected Q margin value and recommendations based on best practices for the Q Margin. 

    """,
    summary_method="reflection_with_llm",
    max_turns=6,
    clear_history=True,
)